In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("./ifeng_data/train.csv", header=None, sep="\t", names=["label", "sentence1"])
df

,label,sentence1
0,政务_文旅,安徽三祖寺发现佛牙舍利 初步判断为宋代皇家所赐
1,科技_数码,员工曝苹果2022年春季发布会要来了！iPhone SE 3即将发布
2,汽车_行业,俄乌战争使大众向中国和美国转移产能
3,科技_车科技,再也不怕事故扯皮！所有新车全面上线黑匣子：比行车记录仪好用
4,体育_中国足球,终身禁赛！体育总局和公安部联手严查严打赌球假球行为
...,...,...
3915,政务_发展治理,住建部约谈五城背后：东莞房价涨幅超深圳、房企南通激战抢地
3916,体育_NBA,欧文回应与哈登不和传言：别把我名字放到这些傻瓜文章里！
3917,政务_发展治理,统计局：中国3月CPI同比增长0.4%，环比下降0.5%
3918,娱乐_电影,她这组写真照真不错


In [5]:
df["sentence1"].str.len().quantile(0.9)

31.0

In [6]:
labels = sorted(df["label"].value_counts().index.tolist())

labels1 = sorted(set([x.split("_")[0] for x in labels]))
labels2 = sorted(set([x.split("_")[1] for x in labels]))
print(labels1, len(labels1))
print(labels2, len(labels2))

['体育', '娱乐', '政务', '时尚', '汽车', '科技'] 6
['5G', 'CBA', 'NBA', '中国足球', '区块链', '反腐', '发展治理', '国际足球', '地方', '导购', '情感', '手机', '政策', '数码', '文旅', '新车', '时装', '明星', '电影', '电视', '美容', '行业', '试驾', '车科技', '音乐'] 25


In [7]:
import sys
print(sys.argv)
sys.argv = sys.argv[:1]

['C:\\Anaconda3\\envs\\transformers\\lib\\site-packages\\ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"41c9c564-d5fd-4c78-a42b-1287e5011fc2"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=C:\\Users\\tzh\\AppData\\Local\\Temp\\tmp-114364DU5BiaeSbG9.json']


In [8]:
import logging
import os

import numpy as np
from transformers import TFAutoModelForSequenceClassification, set_seed, create_optimizer

import tensorflow as tf  # noqa: E402

# import tensorflow.python.keras as keras
import tensorflow.keras as keras


from param_helper import load_params, SavePretrainedCallback
from dataset_helper import load_data, covert_to_tf_dataset, load_data_layer

logger = logging.getLogger(__name__)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # Reduce the amount of console output from TF

In [9]:
model_args, data_args, training_args, checkpoint = load_params("my.json")
datasets, config, is_regression, data_collator, non_label_column_names = load_data_layer(
    model_args, data_args, training_args, checkpoint
)
tf_data = covert_to_tf_dataset(datasets, data_args, training_args, data_collator, non_label_column_names, True)

04/20/2022 22:52:23 - INFO - param_helper - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1

100%|██████████| 2/2 [00:00<00:00, 666.66it/s]

['体育', '娱乐', '政务', '时尚', '汽车', '科技'] 6
['5G', 'CBA', 'NBA', '中国足球', '区块链', '反腐', '发展治理', '国际足球', '地方', '导购', '情感', '手机', '政策', '数码', '文旅', '新车', '时装', '明星', '电影', '电视', '美容', '行业', '试驾', '车科技', '音乐'] 25


04/20/2022 22:52:39 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at C:\Users\tzh\.cache\huggingface\datasets\csv\default-efc3665f38c94ff3\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-680ee6be552e8879.arrow
04/20/2022 22:52:39 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at C:\Users\tzh\.cache\huggingface\datasets\csv\default-efc3665f38c94ff3\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-660798e0b607bc72.arrow


In [10]:
next(iter(tf_data["train"]))

({'attention_mask': <tf.Tensor: shape=(8, 32), dtype=int64, numpy=
  array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
# model_path = model_args.model_name_or_path
# model = TFAutoModelForSequenceClassification.from_pretrained(
#     model_path,
#     config=config,
#     cache_dir=model_args.cache_dir,
#     revision=model_args.model_revision,
#     use_auth_token=True if model_args.use_auth_token else None,
# )
# model

In [12]:
# model.summary()

In [13]:
from model_helper import TFBertForSequenceClassification

model_path = model_args.model_name_or_path
model = TFBertForSequenceClassification.from_pretrained(
    model_path,
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier1', 'classifier2']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  102267648 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 dropout_38 (Dropout)        multiple                  0         
                                                                 
 classifier1 (Dense)         multiple                  4614      
                                                                 
 classifier2 (Dense)         multiple                  19225     
                                                                 
Total params: 102,291,487
Trainable params: 102,291,487
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = keras.optimizers.Adam(
    learning_rate=training_args.learning_rate,
    beta_1=training_args.adam_beta1,
    beta_2=training_args.adam_beta2,
    epsilon=training_args.adam_epsilon,
    clipnorm=training_args.max_grad_norm,
)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn2 = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]
# model.compile(optimizer=optimizer, loss={"output_1": loss_fn, "output_2": loss_fn2}, metrics=metrics)
model.compile(optimizer=optimizer, loss=[loss_fn, loss_fn2], metrics=metrics)

In [16]:
model.fit(
    tf_data["train"],
    validation_data=tf_data["validation"],
    epochs=int(training_args.num_train_epochs),
    callbacks=[],
)

Epoch 1/5
490/490 [==============================] - 89s 144ms/step - loss: 2.2106 - output_1_loss: 0.5397 - output_2_loss: 1.6709 - output_1_accuracy: 0.8260 - output_2_accuracy: 0.4972 - val_loss: 1.5271 - val_output_1_loss: 0.4330 - val_output_2_loss: 1.0940 - val_output_1_accuracy: 0.8582 - val_output_2_accuracy: 0.6490
Epoch 2/5
490/490 [==============================] - 67s 138ms/step - loss: 1.1518 - output_1_loss: 0.2814 - output_2_loss: 0.8705 - output_1_accuracy: 0.9059 - output_2_accuracy: 0.7148 - val_loss: 1.3175 - val_output_1_loss: 0.3923 - val_output_2_loss: 0.9251 - val_output_1_accuracy: 0.8786 - val_output_2_accuracy: 0.7224
Epoch 3/5
490/490 [==============================] - 69s 141ms/step - loss: 0.7692 - output_1_loss: 0.1672 - output_2_loss: 0.6021 - output_1_accuracy: 0.9452 - output_2_accuracy: 0.8066 - val_loss: 1.2946 - val_output_1_loss: 0.3747 - val_output_2_loss: 0.9199 - val_output_1_accuracy: 0.8969 - val_output_2_accuracy: 0.7347
Epoch 4/5
490/490 [===